<a href="https://colab.research.google.com/github/RafaelCaballero/Julio25/blob/main/code/19cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la ciencia de datos con Python

## Regresión lineal con validación cruzada

### Rafael Caballero

Vamos a ver un ejemplo completo de uso de regresión lineal.

#### Paso 0: importar librerías y cargar y preprocesar datos

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold, cross_validate
from sklearn.metrics import mean_squared_error, make_scorer
from math import sqrt
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt


#  Paso 0: Carga y preparación del fichero
import pandas as pd
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/pisaDataClean.csv"
df = pd.read_csv(url)



#### Paso 1 :configuración
Seleccionar las columnas que vamos a utilizar como X y como y

También se puede incluir en estse

In [ ]:
# Paso 1: Dividir las columnas en X e y e.
# Nuestro objetivo será deducir la columna MAT (y) desde dos columnas: SCI y REA, que serán las X
XColumns = ["SCI", "REA"] # una lista de columnas: las X, las características, las features, las dimensiones...
yColumn = "MAT"  # la y, siempre una única columna
# dividimos en dos; X será un dataframe, y una columna (tipo Series)
X = df[XColumns]
y = df[yColumn]


# número de divisiones para validación cruzada, e int. de confianza
repeticiones   = 100  # número de repeticiones del experimento
divisiones = 10  # número de divisiones cada vez

# error a manejar
medidor_error=make_scorer(mean_squared_error, greater_is_better=False)



3.- Declarar el método y entrenar con el conjunto train, obteniendo un modelo

Recordar que esta fase es la más lenta

In [ ]:

repartidor   = RepeatedKFold(n_splits=divisiones, n_repeats=repeticiones)
método = LinearRegression()
cv_res  = cross_validate(método, X, y,
                         cv=repartidor,
                         return_estimator=True,  # devuelve los 1000 modelos!
                         scoring=medidor_error)




4 Cálculo del error

Siemplemente lo obtenemos como la media de los errores obtenidos en cada valor;

si quisiéramos podríamos obtener también un intervalo de confianza para el error

In [ ]:
# ojo porque los valores obtenidos son negativos
mse_neg = cv_res["test_score"]
rmse    = np.sqrt(-mse_neg)
rmse_final = rmse.mean()
print(f"RMSE: {rmse_final:.3f}")

Gráfica de la distribución del error

In [ ]:
plt.hist(rmse, bins=20)
plt.xlabel("RMSE")
plt.ylabel("Frecuencia")
plt.title("Distribución de RMSE en los 200 folds")
plt.show()



### 5 Obtener el modelo final y  grabarlo

Si estamos satisfechos con el resultado ahora entrenamos con el modelo completo

In [ ]:
from sklearn.linear_model import LinearRegression

método = LinearRegression()
modelo_final = método.fit(X, y)

Podemos guardar el resultado

In [ ]:
import joblib

fichero = "modelo_pisa.gz"
joblib.dump(modelo_final, fichero, compress=True)